# Convert DNBs Additional Validation Rules to Patterns

DNBs additional validation rules are available in the 'solvency2-rules' subfolder of the repository.  
The formulas in this file use a specific syntax, this notebook converts this syntax to a syntax that can be interpreted by Python.  
The resulting formulas are called 'patterns'.

## Import packages

In [ ]:
import pandas as pd  # dataframes
from os.path import join # some os dependent functionality
import re # regex
from src import adjust_syntax  # adjust syntax of additional Solvency 2 validation rules
from src import Evaluator  # conversion from 'rules' to pandas expressions for the data-patterns packages

## General parameters

In [ ]:
# Location and name of the file with the additional rules:
RULES_PATH = join('..', 'data', 'downloaded files')
FILENAME_RULES = '2022_02_23_set_aanvullende_controleregels_solvency2.xlsx'

In [ ]:
# Location and names of files with all possible datapoints for QRS and ARS
DATAPOINTS_PATH = join('..', 'data', 'datapoints')
FILENAME_DATAPOINTS_QRS = 'QRS.csv'
FILENAME_DATAPOINTS_ARS = 'ARS.csv'
FILENAME_DATAPOINTS_QRG = 'QRG.csv'
FILENAME_DATAPOINTS_ARG = 'ARG.csv'
FILENAME_DATAPOINTS_QFS = 'QFS.csv'
FILENAME_DATAPOINTS_QFG = 'QFG.csv'
FILENAME_DATAPOINTS_VER = 'VER.csv'

In [ ]:
# Input parameters:
PARAMETERS = {'decimal': 0}
# currently only 'decimal' is available which specifies tolerance during evaluation of patterns.
# decimal: 0 means tolerance = abs(1.5e-0) (= 1.5)

In [ ]:
# # We log to rules.log in the data/instances path
# logging.basicConfig(filename = join(INSTANCES_DATA_PATH, 'rules.log'),level = logging.INFO, 
#                     format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

## Read file with all possible datapoints

We use a simplified taxonomy with all possible datapoints, located in the data/datapoints directory.  
The evaluator uses this taxonomy to generate the patterns.

In [ ]:
# Load files to dataframe:
df_datapoints_qrs = pd.read_csv(join(DATAPOINTS_PATH, FILENAME_DATAPOINTS_QRS), sep=";").fillna("")
df_datapoints_ars = pd.read_csv(join(DATAPOINTS_PATH, FILENAME_DATAPOINTS_ARS), sep=";").fillna("")
df_datapoints_qrg = pd.read_csv(join(DATAPOINTS_PATH, FILENAME_DATAPOINTS_QRG), sep=";").fillna("")
df_datapoints_arg = pd.read_csv(join(DATAPOINTS_PATH, FILENAME_DATAPOINTS_ARG), sep=";").fillna("")
df_datapoints_qfs = pd.read_csv(join(DATAPOINTS_PATH, FILENAME_DATAPOINTS_QFS), sep=";").fillna("")
df_datapoints_qfg = pd.read_csv(join(DATAPOINTS_PATH, FILENAME_DATAPOINTS_QFG), sep=";").fillna("")
df_datapoints_ver = pd.read_csv(join(DATAPOINTS_PATH, FILENAME_DATAPOINTS_VER), sep=";").fillna("")

In [ ]:
# Merge into a single dataframe with all datapoints
df_datapoints = pd.concat([df_datapoints_qrs, df_datapoints_ars, df_datapoints_qrg, df_datapoints_arg,
                           df_datapoints_qfs, df_datapoints_qfg, df_datapoints_ver
                          ], ignore_index = True).drop_duplicates()

In [ ]:
df_datapoints.head()

## Read DNBs Additional Validation Rules

DNBs additional validation rules are currently published as an Excel file on the DNB statistics website. We included the Excel file here in the project.

Here we read the Excel and perform some data cleaning.

In [ ]:
df_rules = pd.read_excel(join(RULES_PATH, FILENAME_RULES), header = 1, engine='openpyxl')
df_rules.drop_duplicates(inplace=True) #remove double lines
df_rules.fillna("", inplace = True)
df_rules = df_rules.set_index('ControleRegelCode')

In [ ]:
df_rules.head()

Adjust syntax of additional Solvency 2 validation rules

In [ ]:
df_rules = adjust_syntax(df_rules)

In [ ]:
df_rules.head()

The Excel file contains rules for different report-types. In the next step we filter out the rules for QRS, ARS, QRG and ARG respectively.

In [ ]:
df_rules_qrs = df_rules.copy()[(df_rules['Standaard'] == 'SOLVENCY') | (df_rules['Standaard'] == 'QRS')]
df_rules_ars = df_rules.copy()[(df_rules['Standaard'] == 'SOLVENCY') | (df_rules['Standaard'] == 'ARS')]

In [ ]:
df_rules_qrs.head()

In [ ]:
df_rules_ars.head()

## Convert the rules to patterns

The evaluator is a piece of Python code, which takes the Additional Validation Rules as input, and transforms it to expressions that can be interpreted by the data_patterns package (patterns).

In [ ]:
evaluator_qrs = Evaluator(df_rules_qrs, df_datapoints, PARAMETERS)

In [ ]:
evaluator_qrs.df_patterns.head()

In [ ]:
evaluator_ars = Evaluator(df_rules_ars, df_datapoints, PARAMETERS)

In [ ]:
evaluator_ars.df_patterns.head()

## Export patterns to rules folder

In [ ]:
evaluator_qrs.df_patterns.to_excel(join('..', 'solvency2-rules', "qrs_patterns_additional_rules.xlsx"))

In [ ]:
evaluator_ars.df_patterns.to_excel(join('..', 'solvency2-rules', "ars_patterns_additional_rules.xlsx"))